# Práctica 1. Resolución de problemas con búsqueda 
## Sistemas Inteligentes 2021/2022

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan 2 problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 



## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros
 
El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 

In [2]:
from search import Problem

In [1]:
# Para que funcione el import el archivo search.py tiene que estar en la mimsa carpeta que este notebook.
# Se puede copiar el archivo search.py a la mimsa carpeta que este notebook.
# Otra opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima/')
import search

In [3]:
## Copiamos aquí la definición de la clase Problem para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [4]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [7]:
p =Jarras()
p.initial

(0, 0)

In [8]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [9]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [10]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [11]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [5]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [6]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [7]:
misioneros.initial

(3, 3, 0)

In [15]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [7]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
    
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        
       ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 

        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0

        return tuple(l)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0 

    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [17]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [18]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco izquierda']

In [19]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

p8.result(p8.initial,"Mover hueco abajo")

In [20]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [21]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco arriba")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [45]:
class Quince_Puzzle(Problem): 
    ## Completa la definición del problema
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,state):
        pos_hueco=state.index(0) # busco la posicion del 0
        actions=list()
        
        if pos_hueco not in (0,1,2,3):
            actions.append("Mover hueco arriba")
        if pos_hueco not in (12,13,14,15):
            actions.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12) or (1,5,9,13):
            actions.append("Mover hueco izquierda")
        if pos_hueco not in (3,7,11,15) or (2,6,10,14):
            actions.append("Mover hueco derecha")
        
        return actions  
    
    def result(self,state,action):
        pos_hueco = state.index(0)
        l = list(state)

        if action == "Mover hueco arriba":
            if pos_hueco-4 > -1:
                l[pos_hueco] = l[pos_hueco-4]
                l[pos_hueco-4] = 0

        elif action == "Mover hueco abajo":
            if pos_hueco+4 < 16:
                l[pos_hueco] = l[pos_hueco+4]
                l[pos_hueco+4] = 0

        elif action == "Mover hueco izquierda":
                if pos_hueco-1 > -1:
                    l[pos_hueco] = l[pos_hueco-1]
                    l[pos_hueco-1] = 0

        elif action == "Mover hueco derecha":
                if pos_hueco+1 < 16:
                    l[pos_hueco] = l[pos_hueco+1]
                    l[pos_hueco+1] = 0

        return tuple(l)

    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0 

    def h(self, node):
        return 1

Realiza algunas pruebas para comprobar que la clase está bien definida. 

In [46]:
p16 = Quince_Puzzle((15, 3, 2, 4, 5, 1, 10, 14, 9, 11, 7, 0, 13, 8, 12, 6))
p16.initial

(15, 3, 2, 4, 5, 1, 10, 14, 9, 11, 7, 0, 13, 8, 12, 6)

In [47]:
p16.actions(p16.initial)

['Mover hueco arriba',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco derecha']

In [48]:
p16.result(p16.initial,"Mover hueco arriba")

(15, 3, 2, 4, 5, 1, 10, 0, 9, 11, 7, 14, 13, 8, 12, 6)

In [49]:
p16.result(p16.initial,"Mover hueco izquierda")

(15, 3, 2, 4, 5, 1, 10, 14, 9, 11, 0, 7, 13, 8, 12, 6)

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [4]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [5]:
# una opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima/')
import search

In [29]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.

In [30]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [31]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [32]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

# breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

In [33]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [34]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [35]:
##breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si es método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## ...............................................................................
##Cómo no tiene control de repetidos, está trabado en un bucle porque sigue bajando y toma ramas sin discriminar si ya pasó por allí

# Añade a la definición del problema Ocho_Puzzle la función solvability que comprueba si un estado inicial tiene solución.
def check_solvability(self, state):
        """ Checks if the given state is solvable """
# The solvability of a configuration can be checked by calculating the Inversion Permutation. 
#If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0        

In [11]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [12]:
sol

'Este estado inicial no tiene solucion'

breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

In [38]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [39]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [54]:
depth_first_graph_search(estado).solution()

['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover h

In [40]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [50]:
estadoDos = Quince_Puzzle((15, 3, 2, 4, 5, 1, 10, 14, 9, 11, 7, 0, 13, 8, 12, 6))

In [51]:
if estadoDos.check_solvability(estadoDos.initial):
    sol= "Este estado inicial tiene solucion"
else: 
    sol="Este estado inicial no tiene solucion"

In [52]:
sol

'Este estado inicial tiene solucion'

In [55]:
breadth_first_tree_search(estadoDos).solution()

In [52]:
breadth_first_graph_search(estadoDos).solution()

KeyboardInterrupt: 

In [43]:
depth_first_graph_search(estadoDos).solution()

KeyboardInterrupt: 

In [44]:
depth_first_tree_search(estadoDos).solution()

KeyboardInterrupt: 

In [12]:
breadth_first_tree_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [14]:
depth_first_tree_search(misioneros).solution()

KeyboardInterrupt: 

In [17]:
depth_first_graph_search(misioneros).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1m', (1, 0)),
 ('1m1c', (1, 1))]

In [16]:
breadth_first_graph_search(misioneros).solution()

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- Misioneros: con los algoritmos de anchura y el algoritmo de profundidad de grafos pudo presentar las soluciones rápidamente, pero no fue lo mismo para el de profundidad de árbol
- Puzle de 8: En la Busqueda de Anchura con Arboles y con Grafo, me da un set de soluciones concreta rápidamente. Sin embargo, usando Primero en Profundidad con Grafo tarda bastante (45 minutos) para llegar al set de soluciones, y con árboles el tiempo era tanto que mi computador hacía crash. Puede ser que entre en un bucle 
- Puzle de 16: Probé que tuviera solución desde el estado que le dí, pero los calculos de cualquier search fueron tan extensos que mi computador crasheaba cuando lo dejaba correr. Es posible que debido a los calculos, inherentemente tarde mas de lo que el ordenador puede soportar, o que entre en un bucle cuando se utiliza los de profundidad por árbol.

**********************************************************************************
Puedes añadir todo el espacio que necesites.

### EJERCICIO 4:  Definición de heurísticas

In [ ]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
Puedes utilizar alguna de las heurísticas vistas en clase. 

# Heurísticas para el problema de los misioneros.
(a,b,c)
h = (a+b)/2;
h = 2(a+b)-c*3


### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [62]:
from search import astar_search, best_first_graph_search, uniform_cost_search

In [57]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [56]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [40]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [41]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [58]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [59]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [60]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [61]:
%%timeit
# timeit -n 1 -r 1
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

6.36 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

23.3 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [69]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

365 ms ± 10.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [70]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

22.8 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.



#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.

Pienso que la mejor heurística viene siendo la Manhattan, considerando que a pesar de tener un resultado de 23.3ms, se ve que tuvo 10 bucles por run, mientras que el Linear tuvo 100. Esto puede impactar el procesamiento en el futuro con una carga mas grande, pero si consideramos unos estados iniciales relativamente sencillos. Se vió en clase que el Linear tiene un menor coste de distancia que el manhattan, y lógicamente se escogería para resolver este tipo de busqueda de estados de solución, pero en el futuro presciento que es mas comendable utilizar el manhattan.

----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [7]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [24]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [20]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [21]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [71]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [14]:
#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas: 
# linear(node) cuenta el número de casillas mal colocadas respecto al estado final.
# manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
# max_heuristic(node: maximo de las dos anteriores
# sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre  de la función. 

import math

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    ln=0
    for (i,j) in zip(node.state, goal):
        if(i!=j):
            ln+=1
    return ln

def manhattan(node):
    cost = 0
    mhd =0
    state = node.state
    goal = (1,2,3,4,5,6,7,8,0)
    for i in state:
        indexI = state.index(i)
        indexJ = goal.index(i)
        cost = (abs((indexJ - indexI))%3)+1
        mhd+=cost
        cost=0
    return mhd
                
def sqrt_manhattan(node):
    #state = node.state
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

In [72]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [73]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [15]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [75]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


In [18]:
##BORRAR ESTO EN EL ENUNCIADO
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
#resuelve_ocho_puzzle(E5,astar_search,manhattan)

In [99]:
resuelve_ocho_puzzle(E5,astar_search,linear)

KeyboardInterrupt: 

In [97]:
resuelve_ocho_puzzle(E5,best_first_graph_search,manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco a

In [21]:
resuelve_ocho_puzzle(E3,depth_first_graph_search)

KeyboardInterrupt: 

In [22]:
resuelve_ocho_puzzle(E1,breadth_first_tree_search)

KeyboardInterrupt: 

### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)

          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 2 | 7 | 3 |   | 3 | 0 | 1 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |    | H | 8 | 7 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 4 | 6 | 8 |   | 8 | 4 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |    | 6 | 5 | 4 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 1 | H | 5 |   | 5 | 2 | 7 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |    | 3 | 2 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)
 
-----------------------------------------------------------------------------------------
                                       E1               E2                  E3              E4          E5
    Anchura                            L=               L= No tiene sol.    L=              L=          L=
                                       T=               T=                  T=              T=          T=
                                       NA=              NA=                 NA=             NA=         NA=
   
    Profundidad                        L=               L= No tiene sol.    L=              L=          L=
                                       T= <123m 55.3s   T=                  T=              T=          T=
                                       NA=              NA=                 NA=             NA=         NA=
                                       
    Coste Uniforme                     L= 17            L= No tiene sol.    L= 20           L= 3        L=
                                       T= 2m 38.5s      T=                  T= 28m 52.6s    T= 0.1s     T= <745m 18.4s
                                       NA= 14105        NA=                 NA= 48428       NA= 13      NA=
                                       
    Primero el mejor (linear)          L= 43            L= No tiene sol.    L= 76           L= 3       L= 28
                                       T= 0.3s          T=                  T= 0.3s         T= 0.8s    T= 0.1s
                                       NA= 689          NA=                 NA= 748         NA= 4      NA= 29
                                                                            
    Primero el mejor (manhattan)       L= 63            L= No tiene sol.    L= 40           L= 3       L= 112
                                       T= 0.1s          T=                  T= 0.1s         T= 0.8s    T= 0.2s
                                       NA= 300          NA=                 NA= 200         NA= 5      NA= 321
                                                                             
    A* (linear)                        L= 17            L= No tiene sol.    L= 20           L= 3        L= 
                                       T= 0.3s          T=                  T= 1m 1.6s      T= 0.1s     T= <379m 13.3s y no terminó
                                       NA= 660          NA=                 NA= 3377        NA= 4       NA= 
                                                                             
    A* (manhattan)                     L= 17            L= No tiene sol.    L= 20           L= 3        L= 28
                                       T= 0.6s          T=                  T= 6.6s         T= 0.9s     T= 10m 6.4s
                                       NA= 1037         NA=                 NA= 3199        NA= 5       NA= 5


 -----------------------------------------------------------------------------------------
PUEDES AMPLIAR LA TABLA CON PRUEBAS ADICIONALES. 


Los de profundidad y de anchura tomaron/tomarán mas tiempo en definir una solución. A pesar de que 4/5 estados si la tengan, en el de profundidad, como no tiene control de repetidos, se mete en un camino infinito. Por otro lado, en el de anchura, este no puede llegar rápidamente a los estados de solución debido a que tiene que ir nivel a nivel revisando los nodos, lo que toma mucho tiempo y coste.

Como el de Coste Uniforme tiene una cola de prioridad y elige los caminos de mismo valor de forma random, toma menos tiempo en la mayoría de los casos; en el caso de E5, asumo que el coste de alguno de sus caminos es igual al de su profundidad, por lo que su cálculo resulta igual al de anchura.

Los de Primero el Mejor tienen el mejor rendimiento. Percibo ya que tecnicamente el puzzle es una tupla, usando las funciones heurísticas Linear y Manhattan dieron lugar a un mejor rendimiento, ya que le primera simplemente cuenta las fichas que estan mal posicionadas como coste, mientras que la segunda cuenta cuanto tomaría para dicha ficha descolocada en llegar a su posición. Ya que se elige el primer nodo que contiene el mejor valor para llegar a dicho coste, entonces el proceso es relativamente rápido.

Los de A* tuvieron el segundo mejor rendimiento, pero debido a que combina anchura con profundidad, me parece que se dió el caso con E5 en Linear donde en algun punto no se cumple la propiedad de consistencia (que la diferencia de las heurísticas del nodo-hijo y nodo-padre era menor o igual que el coste), por lo que afectó el control de repetidos, causando que se metiera en un camino infinito.


 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  



In [63]:
def linearQuince(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    ln=0
    for (i,j) in zip(node.state, goal):
        if(i!=j):
            ln+=1
    return ln

def manhattanQuince(node):
    cost = 0
    mhd =0
    state = node.state
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    for i in state:
        indexI = state.index(i)
        indexJ = goal.index(i)
        cost = (abs((indexJ - indexI))%4)+1
        mhd+=cost
        cost=0
    return mhd
                
def sqrt_manhattanQuince(node):
    #state = node.state
    mhd = manhattan(node)
    return math.sqrt(mhd)

def max_heuristicQuince(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)

In [77]:
QUIN1 = (1,2,3,4,0,6,7,8,9,5,10,11,12,13,14,15)
QUIN2 = (15,3,2,4,5,1,10,14,9,11,7,0,13,8,12,6)

In [58]:
def resuelve_quince_puzzle(estado_inicial, algoritmo, h=None):
    pq=Quince_Puzzle(estado_inicial)
    pqp=Problema_con_Analizados(pq)
    if pq.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(pqp,h).solution()
        else: 
            sol= algoritmo(pqp).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),pqp.analizados))
    else: 
        print("Este problema no tiene solucion. ")

In [73]:
resuelve_quince_puzzle(QUIN1,astar_search,linearQuince)

KeyboardInterrupt: 

In [74]:
resuelve_quince_puzzle(QUIN1,astar_search,manhattanQuince)

KeyboardInterrupt: 

In [69]:
resuelve_quince_puzzle(QUIN1,best_first_graph_search,linearQuince)

Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover 

In [78]:
resuelve_quince_puzzle(QUIN2,best_first_graph_search,linearQuince)

KeyboardInterrupt: 

In [71]:
resuelve_quince_puzzle(QUIN1,best_first_graph_search,manhattanQuince)

KeyboardInterrupt: 

In [75]:
resuelve_quince_puzzle(QUIN1,uniform_cost_search)

KeyboardInterrupt: 

### Ejercicio 7:  resolver usando distintas búsquedas y/o heurísticas, el problema del 16 puzzle para algunos  estados iniciales:
QUIN1 = (1,2,3,4,0,6,7,8,9,5,10,11,12,13,14,15)
QUIN2 = (15,3,2,4,5,1,10,14,9,11,7,0,13,8,12,6)

 
-----------------------------------------------------------------------------------------
                                       QUIN1            QUIN2                          
    Anchura                            L=               L=                               
                                       T=               T=                               
                                       NA=              NA=                           
   
    Profundidad                        L=               L=                             
                                       T=               T=                              
                                       NA=              NA=                             
                                       
    Coste Uniforme                     L=               L=                               
                                       T=               T=                                
                                       NA=              NA=                            
                                       
    Primero el mejor (linear)          L= 61            L=                               
                                       T= 1.0s          T=                                
                                       NA= 538          NA=                              

    Primero el mejor (manhattan)       L=               L=                               
                                       T=               T=                                
                                       NA=              NA=                              
                                                                             
    A* (linear)                        L=               L=                                
                                       T=               T=                                
                                       NA=              NA=                              
                                                                             
    A* (manhattan)                     L=               L=                                
                                       T=               T=                                
                                       NA=              NA=                             


 -----------------------------------------------------------------------------------------

Escribe aquí tus conclusiones sobre los resultados y la eficiencia de las distintas opciones. 
No puedo comentar mucho, ya que el tiempo de procesamiento de cada función heurística y algoritmos toman mucho tiempo, como se advirtió. Utilizando Primero el Mejor con Linear en el estado de inicio mas simple, se obtubo un resultado inmediato. Por otro lado, los demás casos toman un tiempo de procesamiento mayor a 25 minutos, lo que se atribuye a varios factores: 

1) Esencialmente se han duplicado el monto de posiciones disponibles al igual que el posicionamiento (ya no es 3x3, sino 4x4), por lo que tiene que analizar muchos mas nodos, caminos, y coste que antes (lo que ya tomaba mucho tiempo). 

2) Debido a que hay un monto de "estados" mas frecuente, es mas fácil que los algoritmos que no toman en consecuencia el control de repetidos caigan en un camino infinito.

4) Utilizando la heurística Manhattan, esta debe ahora considerar muchos mas valores/posiciones que antes, al igual que calcular el trayecto/coste de cada uno a su posición deseada dentro de un esquema mucho mas amplio, por lo que puede haber inconsitencias. Esto, combinado con lo ya mencionado de los algoritmos, hace que la resolución de los estados no parezca "óptima" al momento de compararlo con el Puzzle de 8, sin embargo disponiendo de un computador con un mejor nivel de procesamiento y disponer de minimo 30 minutos en adelante para cada uno, se podría llegar a los estados deseados. 


###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 


a) Pude definir:

    Altura Maxima = 3
    Numero de Muelles = 2
    Numero de Pilas = 3 x Muelle
        > (1,2,3,4,5,6)
    Numero de Contenedores = 11
        > ("C01 ","C02 ","C03 ","C04 ","C05 ","C06 ","C07 ","C08 ","C09 ","C10","C11")

    Objetivo: Dejar los Contenedores Objetivos en el Muelle 1

    Acciones:
        > Se supone que hay dos gruas que mueven los contenedores ya sea entre sus propias pilas o los ponen en una cinta para que la contigua los acomode en las suyas. Al momento de escribir el código, tanto la prescencia de las gruas como de el tipo de cinta son negligibles; solo se necesita el nombre del contenedor, a donde va, y si fue por cinta, ya que dentro de la implementación se define a que muelle pertenecen

        1) "Mover Ci a Pila j"
        2) "Mover Ci a Pila j por Cinta"

        > Y, acorde a las reglas establecidas, sólo se necesitarán 2 operadores:
    
        1) moverObjetivo()
        2) moverNoObjetivo()

    Estados:
        > Decidí declarar los estados como una tupla de tuplas (muelles) de tuplas (pilas), de forma que si se quiere insertar otro muelle en el futuro, es mas fácil de realizar. Sin embargo, tuve que considerar que una pila puede tener espacios, ya que puede tener 1 o 2 contenedores solamente. Por esto, decidí incluir espacios en blanco [''] para representar estos

        > Los estados son:

        Estado Inicial Propuesto = (( ("C07", "C01", ""),("C10", "C09", ""),("C11","","") ),( ("C08", "C04", ""),("C02", "C03", "C05"),("C06","","") ))
        Estado Final Propuesto   = (( ("C01", "C02", "C03"),("C04", "C05", ""),("C06","","") ),(// No importa, ya que nos enfocámos en el Muelle 1))


    Heurística/Coste:
        >Decidí optar por un coste para cada acción de acuerdo a su naturaleza:

        1) "Mover Ci a Pila j" => Tiene un coste de 1   [Ya que levanta y acomoda en sus propias pilas]
        1) "Mover Ci a Pila j por Cinta" => Tiene un coste de 2     [Ya que una grua tiene que levantar de una pila y soltar en la cinta, mientras que otra la recoge de esta y la acomoda en una de sus pilas]




b) Algoritmos vistos en Clase:

    Búsqueda Ciega:
        > Para una búsqueda ciega, utilizaría los de Coste Uniforme y Profundidad Iterativa

            1) Coste Uniforme: este sería mi primera opción porque en un principio trata de llegar a cada estado en el menor coste posible, y a pesar de que no tiene control de repetidos, en este caso vale la pena tomar el riesgo ya que es posible que un estado se vuelva a presentar mientras se acomodan de nuevo, pero tomando en cuenta de que en total hay 18 posiciones diferentes (3 pilas con un límite de 3 contenedores) y solo hay 11 contenedores, es menos probable que un estado repetido aparezca frecuentemente.

            2) Profundidad Iterativa: sería mi segunda opción, ya que el espacio de estados es grande y hay muchas posibilidades de posicionar los contenedores, sería factible chequear cada posible chequear los mejores estados tanto en profundidad como en anchura. De nuevo, como mencioné anteriormente, el control de repetidos no tiene que ser tan estricto o existente.

    Búsqueda Heurística:
        > Para la búsqueda heurística, utilizaría los A* (con Manhattan) y Primero el Mejor (también con Manhattan)

            1) A*(Manhattan): Este sería mi primera opción ya que estima el coste mínimo utilizando la distancia Manhattan, la cual me parece mejor usar que la lineal considerando de que en el contexto del problema, las gruas tienen que "mover" los contenedores, por lo que el calcular cuanto le tomaría a uno llegar a su posición ideal prueba ser más acertado que solo contar las mal colocadas (ademas que solo vamos a tomar en cuenta un solo muelle).

            2) Primero el Mejor(Manhattan): Este sería mi segunda opción porque considerando que se tienen que acomodar varios contenedores, sería mas óptimo irse por el que parezca "mejor" a los demás y así no perder tiempo analizando todos los nodos disponibles. 
    

In [220]:

class Puerto(Problem):

    global PHeight
    global Containers
    global Piles
    global D1Piles
    global D2Piles
    global Objectives
    global DockNum
    global PileNum

    def __init__(self, initial, goal=(("C01","C02","C03"),("C04","C05",''),("C06",'','')) ):
        self.goal = goal
        self.PHeight = 3 
        self.Containers = ("C01","C02","C03","C04","C05","C06","C07","C08","C09","C10","C11")
        self.Piles = (1,2,3,4,5,6)
        self.D1Piles=(1,2,3)
        self.D2Piles=(4,5,6)
        self.Objectives = []
        self.DockNum = 2
        self.PileNum = 3
 
        for pila in goal:
            for i in range(self.PHeight):
                if pila[i] != '':
                    self.Objectives.append(pila[i])
        Problem.__init__(self, initial, goal)

    
    def espacioEnPila(self, estado, numPila):
        for dock in estado:
            for pila in dock:
                if dock.index(pila) != numPila:
                    if '' in pila:
                        return [estado.index(dock), dock.index(pila)]
        return False
                        
    def espacioEnPilaNoObj(self, estado, numPila):
        for dock in estado:
            for pila in dock:
                if dock.index(pila) != numPila:
                    if '' in pila:
                        if pila[0] not in self.Objectives and pila[1] not in self.Objectives:
                            return [estado.index(dock), dock.index(pila)]
        return False


    def moverObjetivo(self, estado, contenedor, index, dockOfPila, numPila):
        for pila in self.goal:
            for c in pila:
                if c == contenedor and c != '':
                    if pila.index(c) != index:
                        avail = self.espacioEnPila(estado, numPila)
                        if avail:
                            if avail[0] == dockOfPila:
                                pila = avail[1] + 1
                                return "Mover " + str(contenedor) + " a Pila " + str(pila)
                            else:
                                pila = avail[1] + 4
                                return "Mover " + str(contenedor) + " a Pila " + str(pila) + "por Cinta"
        return False

    def moverNoObjetivo(self, estado, contenedor, dockOfPila, numPila):
        avail = self.espacioEnPilaNoObj(estado, numPila)
        if avail:
            if avail[0] == dockOfPila:
                pila = avail[1] + 1
                return "Mover " + str(contenedor) + " a Pila " + str(pila)
            else:
                pila = avail[1] + 4
                return "Mover " + str(contenedor) + " a Pila " + str(pila) + " por Cinta"
        else:
            return False


    def actions(self,estado):
        accs=list()
        for dock in estado:
            for pila in dock:
                if pila[0] != '':
                    pl = list(pila)
                    contenedor = pl.pop()
                    while pl:
                        numPila = dock.index(pila)
                        dockOfPila = estado.index(dock)
                        if contenedor in self.Objectives:
                            indexOfCont = pila.index(contenedor)
                            action = self.moverObjetivo(estado, contenedor, indexOfCont, dockOfPila, numPila)
                            if action:
                                accs.append(action)
                                return accs
                        elif contenedor != '':
                            action = self.moverNoObjetivo(estado, contenedor, dockOfPila, numPila)
                            if action:
                                accs.append(action)
                                return accs
                        contenedor = pl.pop()
        return []


    def takeAwayContainer(self, dock, pile, c, state):
        i = 0
        for x in range(self.PHeight-1,-1,-1):
            if state[dock][pile][x] == c:
                i = x
                break
        con = state[dock][pile][i]
        state[dock][pile] = state[dock][pile][:i]+ [''] + state[dock][pile][i+1:]
        return con

    def getContainer(self,c, state):
        for dock in state:
            for pila in dock:
                if c in pila:
                    return self.takeAwayContainer(state.index(dock),dock.index(pila),c,state)

    def accommodateContainer(self, dock, pile, c, state):
        i = self.PHeight
        for x in range(self.PHeight-1,-1,-1):
            if state[dock][pile][x] == '':
                i = i - 1
        state[dock][pile] = state[dock][pile][:i]+ [c] + state[dock][pile][i+1:]


    def placeContainer(self, c, state, p):
        dockChosen = -1
        pileChosen = -1
        if p in self.D1Piles:
            dockChosen = 0
            pileChosen = p-1
        elif p in self.D2Piles:
            dockChosen = 1
            pileChosen = p-4
        if pileChosen != -1 and dockChosen != -1:
            self.accommodateContainer(dockChosen,pileChosen,c,state)

    def initList(self,estado):
        l = []        
        for i in range(self.DockNum):
            l.append([])
            for j in range(self.PileNum):
                l[i].append(list(estado[i][j]))
        return l
    
    def listToTuple(self, estado):
        t = []
        for i in range(self.DockNum):
            auxT = []
            for j in range(self.PileNum):
                auxT.append(tuple(estado[i][j]))
            t.append(tuple(auxT))
        return tuple(t)

    def result(self,estado,accion):

        l = self.initList(estado)
        c = accion[6:9]
        p = int(accion[17:19].rstrip())

        if c in self.Containers and p in self.Piles:
            cont = self.getContainer(c, l)
            self.placeContainer(cont, l, p)
        t = self.listToTuple(l)
        return t
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0 

    def coste_de_aplicar_accion(self, estado, accion):
        if "Mover" and "a Pila" in accion:
            if "por Cinta" in accion:
                return 2
            return 1

In [221]:
#Manhattan Muelle
def manhattanDock(node):
    cost = 0
    mhd =0
    state = node.state
    goal = (("C01","C02","C03"),("C04","C05",''),("C06",'',''))
    DockNum = 2
    PileNum = 3
    PHeight = 3

    for pile in goal:
        for container in pile:
            cost=0
            x=0
            found = False
            while x < DockNum and not found:
                y=0
                while y < PileNum and not found:
                    w = 0
                    while w < PHeight and not found:
                        if container != state[x][y][w]:
                            cost+=1
                            w+=1
                        else:
                            found = True
                    if not found:
                        y+=1
                if not found:
                    x+=1
            mhd+=cost
    return mhd

In [211]:
Initial = ( (("C07","C01","") , ("C10","C09","") , ("C11","","")) , (("C08","C04","") , ("C02","C03","C05") , ("C06","","")) )

In [194]:
Initial

((('C07', 'C01', ''), ('C10', 'C09', ''), ('C11', '', '')),
 (('C08', 'C04', ''), ('C02', 'C03', 'C05'), ('C06', '', '')))

#Cambié brevemente la definición del node de manhattanDock para que simplemente pasase el estado inicial tal cual come venía. Al terminar los cálculos, el coste fue de 71

In [60]:
def resuelve_puerto(estado_inicial, algoritmo, h=None):

    pu=Puerto(estado_inicial)
    pup=Problema_con_Analizados(pu)
    if pu.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(pup,h).solution()
        else: 
            sol= algoritmo(pup).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),pup.analizados))
    else: 
        print("Este problema no tiene solucion. ")

In [222]:
m = Puerto(Initial)
m.initial

((('C07', 'C01', ''), ('C10', 'C09', ''), ('C11', '', '')),
 (('C08', 'C04', ''), ('C02', 'C03', 'C05'), ('C06', '', '')))

In [223]:
act = m.actions(m.initial)
act

['Mover C01 a Pila 2']

In [224]:

print(m.initial)
lm = m.initList(Initial)
print(lm)
con = m.getContainer("C01", lm)
print(con)
print(lm)
m.placeContainer(con, lm, 2)
print(lm)
t = m.listToTuple(lm)
print(t)


((('C07', 'C01', ''), ('C10', 'C09', ''), ('C11', '', '')), (('C08', 'C04', ''), ('C02', 'C03', 'C05'), ('C06', '', '')))
[[['C07', 'C01', ''], ['C10', 'C09', ''], ['C11', '', '']], [['C08', 'C04', ''], ['C02', 'C03', 'C05'], ['C06', '', '']]]
C01
[[['C07', '', ''], ['C10', 'C09', ''], ['C11', '', '']], [['C08', 'C04', ''], ['C02', 'C03', 'C05'], ['C06', '', '']]]
[[['C07', '', ''], ['C10', 'C09', 'C01'], ['C11', '', '']], [['C08', 'C04', ''], ['C02', 'C03', 'C05'], ['C06', '', '']]]
((('C07', '', ''), ('C10', 'C09', 'C01'), ('C11', '', '')), (('C08', 'C04', ''), ('C02', 'C03', 'C05'), ('C06', '', '')))


In [225]:
tp = m.result(m.initial, 'Mover C01 a Pila 2')
print(tp)

((('C07', '', ''), ('C10', 'C09', 'C01'), ('C11', '', '')), (('C08', 'C04', ''), ('C02', 'C03', 'C05'), ('C06', '', '')))


In [226]:
print(m.result(m.initial,"Mover C01 a Pila 2"))

((('C07', '', ''), ('C10', 'C09', 'C01'), ('C11', '', '')), (('C08', 'C04', ''), ('C02', 'C03', 'C05'), ('C06', '', '')))


No comprendo el error obtenido (abajo), ya que se refiere a un NoneType que no tiene atributos. Sin embargo, de nuevo, puedo inferir que la búsqueda A* con Manhattan tendría mejores resultados mas no mas "rápidos" que el coste uniforme. Considerando que, el coste de manhattan de el estado inicial debería de amontonar a 71, y que el espacio de estados es muy variado considerando que activamente se deben de reoorganizar los contenedores de tal forma que se llegue al estado final lo más optimamente posible, el A* permite efectuar esto al mismo tiempo que tener la propiedad de "abandonar" los caminos que no parecen favorables y optar por el mejor. Por lo que, este se ajusta mejor al momento de sacar las diferentes combinaciones, igualmente considerando que el espacio de estados puede incrementar si otro muelle es añadido o si la altura maxima de cada pila es incrementada.

In [228]:

#Heurística
resuelve_puerto(Initial,astar_search,manhattanDock)

AttributeError: 'NoneType' object has no attribute 'solution'

In [230]:
resuelve_puerto(Initial,uniform_cost_search)

AttributeError: 'NoneType' object has no attribute 'solution'

In [231]:
astar_search(Puerto(Initial), manhattanDock).solution()

AttributeError: 'NoneType' object has no attribute 'solution'